In [4]:
using JLD2
using ITensors
using ITensorMPS
using LinearAlgebra

In [5]:
include("../src/main.jl")
include("../src/observables.jl")
include("../src/states.jl")
include("../src/singled_measurements.jl")
include("../src/doubled_measurements.jl")
include("../src/circuits.jl")

pure_sample (generic function with 1 method)

In [ ]:
function MPS_neelstate(L::Int, K::Int)
    N = L
    sites = siteinds("Qubit", N)
    ρ = MPS(sites, i -> mod(i-1+L,2)==0 ? "0" : "1")
    return ρ, sites
end

In [52]:
function get_bit(n::Int, i::Int)
    return (n >> (i-1)) & 1
end

function MPS_bitstring(n::Int, sites::Vector{Index{Int64}})
    return MPS(sites, i -> get_bit(n,i)==0 ? "0" : "1")
end

function MPS_bitstring_dm(n1::Int, n2::Int, sites::Vector{Index{Int64}})
    f(i) = mod(i,2) == 1 ? (get_bit(n1,i÷2+1)==0 ? "0" : "1") : (get_bit(n2,i÷2)==0 ? "0" : "1")
    return MPS(sites, f)
end

function bitstrings(L::Int, n::Int)
    return [k for k in 0:(2^L-1) if count_ones(k) == n]
end

bitstrings (generic function with 1 method)

In [73]:
function condensate(L::Int, n::Int; cutoff=1e-8, maxdim=200)
    N = 2L
    sites = siteinds("Qubit", N)
    product_states = bitstrings(L, n)
    ρs = [MPS_bitstring_dm(n1, n2, sites) for n1 in product_states for n2 in product_states]
    
    ρ = copy(ρs[1])
    for k in 2:length(ρs)
        ρ += ρs[k]
        truncate!(ρ; cutoff=cutoff, maxdim=maxdim)
    end
    return ρ, sites
end

condensate (generic function with 1 method)

In [80]:
function old_condensate(L::Int, n::Int; cutoff=1e-8, maxdim=200)
    N = 2L
    sites = siteinds("Qubit", N)
    product_states = bitstrings(L, n)
    ρs = [MPS_bitstring_dm(n1, n2, sites) for n1 in product_states for n2 in product_states]
    
    ρ = reduce(+, ρs)
    return ρ, sites
end

old_condensate (generic function with 1 method)

In [85]:
L = 10
N = L÷2
ρ, sites = condensate(L, N; cutoff=1e-8, maxdim=200);

In [86]:
L = 12
N = L÷2
ρ, sites = old_condensate(L, N);

InterruptException: InterruptException:

In [70]:
maxlinkdim(ρ)

25

In [69]:
truncate!(ρ; maxdim=200, cutoff=1e-8)

MPS
[1] ((dim=2|id=536|"Qubit,Site,n=1"), (dim=2|id=643|"Link,l=1"))
[2] ((dim=2|id=939|"Qubit,Site,n=2"), (dim=4|id=664|"Link,l=2"), (dim=2|id=643|"Link,l=1"))
[3] ((dim=2|id=206|"Qubit,Site,n=3"), (dim=6|id=873|"Link,l=3"), (dim=4|id=664|"Link,l=2"))
[4] ((dim=2|id=910|"Qubit,Site,n=4"), (dim=9|id=0|"Link,l=4"), (dim=6|id=873|"Link,l=3"))
[5] ((dim=2|id=141|"Qubit,Site,n=5"), (dim=12|id=905|"Link,l=5"), (dim=9|id=0|"Link,l=4"))
[6] ((dim=2|id=173|"Qubit,Site,n=6"), (dim=16|id=941|"Link,l=6"), (dim=12|id=905|"Link,l=5"))
[7] ((dim=2|id=665|"Qubit,Site,n=7"), (dim=20|id=142|"Link,l=7"), (dim=16|id=941|"Link,l=6"))
[8] ((dim=2|id=298|"Qubit,Site,n=8"), (dim=25|id=486|"Link,l=8"), (dim=20|id=142|"Link,l=7"))
[9] ((dim=2|id=797|"Qubit,Site,n=9"), (dim=20|id=178|"Link,l=9"), (dim=25|id=486|"Link,l=8"))
[10] ((dim=2|id=71|"Qubit,Site,n=10"), (dim=16|id=901|"Link,l=10"), (dim=20|id=178|"Link,l=9"))
[11] ((dim=2|id=885|"Qubit,Site,n=11"), (dim=12|id=679|"Link,l=11"), (dim=16|id=901|"Link,l=10

In [35]:
MPS_bitstring_dm(5, 5, sites)

MPS
[1] ((dim=2|id=623|"Qubit,Site,n=1"), (dim=1|id=337|"Link,l=1"))
[2] ((dim=1|id=337|"Link,l=1"), (dim=2|id=413|"Qubit,Site,n=2"), (dim=1|id=655|"Link,l=2"))
[3] ((dim=1|id=655|"Link,l=2"), (dim=2|id=780|"Qubit,Site,n=3"), (dim=1|id=892|"Link,l=3"))
[4] ((dim=1|id=892|"Link,l=3"), (dim=2|id=233|"Qubit,Site,n=4"), (dim=1|id=386|"Link,l=4"))
[5] ((dim=1|id=386|"Link,l=4"), (dim=2|id=216|"Qubit,Site,n=5"), (dim=1|id=457|"Link,l=5"))
[6] ((dim=1|id=457|"Link,l=5"), (dim=2|id=794|"Qubit,Site,n=6"), (dim=1|id=226|"Link,l=6"))
[7] ((dim=1|id=226|"Link,l=6"), (dim=2|id=963|"Qubit,Site,n=7"), (dim=1|id=57|"Link,l=7"))
[8] ((dim=1|id=57|"Link,l=7"), (dim=2|id=391|"Qubit,Site,n=8"), (dim=1|id=613|"Link,l=8"))
[9] ((dim=1|id=613|"Link,l=8"), (dim=2|id=678|"Qubit,Site,n=9"), (dim=1|id=689|"Link,l=9"))
[10] ((dim=1|id=689|"Link,l=9"), (dim=2|id=758|"Qubit,Site,n=10"))


In [18]:
get_bit(5,2)

0

In [10]:
count_ones(3)

2

In [6]:
sites = siteinds("Qubit", 10)

10-element Vector{Index{Int64}}:
 (dim=2|id=623|"Qubit,Site,n=1")
 (dim=2|id=413|"Qubit,Site,n=2")
 (dim=2|id=780|"Qubit,Site,n=3")
 (dim=2|id=233|"Qubit,Site,n=4")
 (dim=2|id=216|"Qubit,Site,n=5")
 (dim=2|id=794|"Qubit,Site,n=6")
 (dim=2|id=963|"Qubit,Site,n=7")
 (dim=2|id=391|"Qubit,Site,n=8")
 (dim=2|id=678|"Qubit,Site,n=9")
 (dim=2|id=758|"Qubit,Site,n=10")

In [ ]:
MPS(sites,)

ArgumentError: ArgumentError: Overload of "state" or "state!" functions not found for state name "0000000000" and Index tags "Qubit,Site,n=1"